In [4]:
import time

import numpy as np
import epics

%gui qt5
%matplotlib qt5

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
from matplotlib import rcParams
rcParams.update(
    {'font.size': 14,'lines.linewidth': 2, 'axes.grid': True})

from siriuspy.devices import BunchbyBunch

In [5]:
def plot_sweep_results(ctrl_values, mon_values, ctrl_label=None, mon_label=None):
    f = plt.figure(figsize=(9,5))
    gs = mpl_gs.GridSpec(1, 1)
    gs.update(
        left=0.15, right=0.98, bottom=0.12, top=0.90,
        hspace=0.01, wspace=0.1)
    ax = plt.subplot(gs[0, 0])
    ax.plot(ctrl_values, mon_values)
    if ctrl_pvname:
        ax.set_xlabel(ctrl_pvname)
    if mon_pvname:
        ax.set_ylabel(mon_pvname)
    f.show()
    return f, ax

In [6]:
bbb = BunchbyBunch('L')

In [ ]:
bbb.connected

# Longitudinal Feedback Timing Setup

The process of adjusting longitudinal feedback settings for optimal front-end beam signal detection and best back-end performance is historically called “feedback timing”.
In this application note I will present a step-by-step procedure for timing the longitudinal feedback system based on an iGp baseband processor and FBE longitudinal front/back unit.

In this process we are trying to achieve several goals:

 - Adjust ADC sampling clock relative to the beam signal;
 - Adjust front-end carrier phase to achieve phase detection;
 - Adjust DAC clock to optimize kick signal for the right bunch;
 - Adjust back-end carrier phase to maximize the kick voltage.

Note that there is certain symmetry in this procedure - we are adjusting one clock and one carrier phase in both front and back-ends.

Some of the steps below are only needed during initial installation or after hardware reconfiguration, e.g. cable/attenuator/amplifier replacement or software/firmware/gateware upgrade.

## Step 1: Rough front-end timing

At this point the setting of the front-end phase shifter is arbitrary, so the mixer can be anywhere from amplitude to phase detection.
We would like to adjust the phase shifter for amplitude detection and then roughly time the ADC.
This is necessarily an iterative process where we alternate between adjusting the ADC delay and the phase shifter setting.
Typically one can see the effect of the phase shifter adjustment pretty much at any ADC delay - so we start from phase shifter adjustment.

Move the phase shifter in steps of 100 (each step of the phase shifter DAC is ~0.25 degrees) until the single-bunch spike amplitude is maximized
(sign is unimportant) on the mean plot in the waveform window. This step can be automated by the sweep script as shown here.
Roughly time the front-end by moving the ADC delay in steps of 500 ps and observing the response in the waveform window.
We are looking for maximum mean amplitude of the single-bunch spike. This step can also be automated.

1- Move the phase shifter in steps of 50 (each step of the phase shifter DAC is ~0.25 degrees) until the single-bunch spike amplitude is maximized (sign is unimportant) on the mean plot in the waveform window.

In [ ]:
phases = np.arange(0, 2000, 50)
mean_values = bbb.sweep_phase_shifter(phases, delay=2)

In [ ]:
bunch_number = 1
vals = mean_values[:, bunch_number - 1]
plot_sweep_results(phases, vals, ctrl_label='Servo Phase', mon_label='SRAM Mean')

set the maximum value:

In [ ]:
bbb.fbe.z_phase = phases[np.argmax(vals)]

2- Roughly time the front-end by moving the ADC delay in steps of 500 ps and observing the response in the waveform window. We are looking for maximum mean amplitude of the single-bunch spike.

In [ ]:
adc_delays = np.arange(0, 1500, 100)
mean_values = bbb.sweep_adc_delay(adc_delays, delay=2)

In [ ]:
bunch_number = 1
vals = mean_values[:, bunch_number - 1]
plot_sweep_results(adc_delays, vals, ctrl_label='ADC Delay', mon_label='SRAM Mean')

set the maximum value:

In [ ]:
bbb.timing.adc_delay = adc_delays[np.argmax(vals)]

3- Repeat the two steps above after the first pass.

## Step 2: Adjusting fiducial delay

***only needed during initial installation or after hardware reconfiguration***

At this point we need to adjust FIDUCIAL DELAY in the timing panel. To do so, determine which bucket is seen as filled on the waveform plots (Nwfm).
Let's suppose the actual filled bucket number is Nact. Then FIDUCIAL DELAY must be increased by (Nwfm-Nact)/2.
The resultant value for FIDUCIAL DELAY should be less than h/2 where h is the harmonic number. If the value is larger than or equal to h/2, subtract h/2 from it.
This adjustment has granularity of two buckets. If the result is one bucket off, increase FIDUCIAL SIGNAL OFFSET by one RF period.

## Step 3: Setting front-end for phase detection

Now we need to set up the front-end for phase detection. Adjust the front-end phase shifter so that the mean of the filled bunch is the same as that of the empty buckets.

Here we are sweeping the front-end phase shifter from 0 to 2000 with steps of 50 and recording the mean value of bunch 1.

From this sweep one can determine phase shifter setting that produces phase-detection (no offset in the filled bucket relative to the empty buckets).


In [ ]:
phases = np.linspace(1500, 1650, 21)
mean_values = bbb.sweep_phase_shifter(phases, wait=2)

In [ ]:
bunch_number = 1
plot_sweep_results(phases, mean_values, ctrl_label='Servo Phase', mon_label='SRAM Mean')

In [ ]:
slc = slice(bunch_number-1 +10, bunch_number-1 -10)
values = mean_values - mean_values[:, slc].mean(axis=1)[:, None]
norm_values = np.abs(values)

norms = norm_values[:, bunch_number-1]
min_val = phases[np.argmin(norms)]
print(min_val)

In [ ]:
bbb.fbe.z_phase = min_val

## Step 4: Determining the synchrotron frequency

As a starting point one should use the nominal value known from previous measurement and/or machine parameters.

After steps 1 and 2 it might be possible to see the synchrotron peak in the averaged spectrum plot.

Use RECORD LENGTH and REC. DOWNSAMPLE settings to extend data acquisition time span and, therefore, achieve finer frequency resolution.

## Step 5: Turn on the drive

Drive panelNow we need to drive the beam at the synchrotron frequency. Open the drive panel and set it up as follows:

 - Set DRIVE ENABLE to “DRIVE”
 - Set DRIVE MODE to “Turn-by-turn”
 - Type synchrotron requency determined in step 3 in the FREQUENCY field
 - Set WAVEFORM SELECTION to “Sine”
 - Set AMPLITUDE to 1
 - Set DRIVE PATTERN to all bunches. For example, if harmonic number is 232, type “1:232”

## Step 6: Optimizing back-end carrier phase

At this point in the procedure you should observe some beam motion in the filled bucket, driven by the excitation we have applied.

It might be useful to fine-tune the drive frequency a bit to improve the response.

We are looking at the RMS plot on the waveform panel - the RMS of the filled bunch should be visibly above that of the empty buckets.

Adjust back-end phase shifter to maximize that RMS value.

In [ ]:
be_phases = np.linspace(200, 750, 50)
rms_values = bbb.sweep_backend_phase(be_phases, wait=5)

In [ ]:
plot_sweep_results(
    be_phases, rms_values,
    ctrl_label='Backend Phase', mon_label='SRAM Peak Amplitude')

set the maximum value:

In [ ]:
bbb.fbe.be_phase = delays[np.argmax(rms_values)]

## Step 7: Determining back-end bucket offset

***only needed during initial installation or after hardware reconfiguration***

At this point in the process the system should be exciting the beam strongly at the synchrotron frequency.
The difference between the excited and the normal states can be checked by flipping DRIVE ENABLE back to “FEEDBACK” for a few seconds.

Now we need to determine which channel in the back-end is actually kicking the beam. To do that we perform a binary search.
Initially we are driving all buckets, say 1:232. Now we will change the DRIVE PATTERN to only drive one half of the previous range, that is 1:116.
If the beam is still driven, then continue dividing the current range, otherwise switch to the other half, i.e. 117:232.
Using this method after a few steps (8 for 232) you will identify a single bucket Nm which, when enabled in the DRIVE PATTERN, excites the beam.
Now, adjust OUTPUT DELAY on the timing panel by Nm-Nact, where Nact is the actual filled bucket number. If the delta is negative, add harmonic number.
After this adjustment, setting the DRIVE PATTERN to Nact should drive the beam.

## Step 8: DAC clock timing

Next step is to time the DAC output relative to the beam. We will start from maximizing the excitation response while adjusting the DAC DELAY on the timing panel.

The sweep script is very useful during this step - see an example. Reasonable step size during this adjustment is 100 ps.

During this adjustment it might be necessary to lower the drive amplitude if the front-end is being saturated - typically manifested by extended flat-top response.
The drive amplitude should be lowered until the RMS at the top of the sweep starts to drop.

Note that maximizing the response might require changes to the OUTPUT DELAY value.
If in your sweep the RMS continues to increase at the endpoint (0 or TRF), you need to extend the sweep further.
Back-end timing adjustment has two controls“ DAC DELAY with the range of one RF period and 10 ps step and OUTPUT DELAY with the range of one turn and one RF period step.
Take a system with 2 ns RF period as an example. Then setting of OUTPUT DELAY of N and DAC DELAY of 1990 ps is 10 ps away from OUTPUT DELAY of N+1 and DAC DELAY of 0 ps.

Once the RMS response is maximized, we can fine-tune the timing. This method has been proposed by Alessandro Drago of LNF-INFN.
The idea is to equalize the parasitic excitation of the neighboring buckets. Set the AMPLITUDE on the drive panel to 1.
Then set the DRIVE PATTERN to two values: first to Nact-1 and then to Nact+1. Note the RMS levels at each of these settings.
If the reading at Nact-1 is larger, increase DAC DELAY in small (10 ps) steps. Otherwise, reduce DAC DELAY.
The goal is to equalize the coupling to the two buckets adjacent to the driven one.

In [ ]:
dac_delays = np.linspace(1500, 1950, 20)
rms_values864 = bbb.sweep_dac_delay(dac_delays, wait=10)

In [ ]:
rms_values002 = bbb.sweep_dac_delay(dac_delays, wait=10)

In [ ]:
rms_values = np.array([rms_values864, rms_values002]).T
plot_sweep_results(
    dac_delays, rms_values, ctrl_label='DAC Delay', mon_label='SRAM Peak Amplitude')

set the maximum value:

In [ ]:
bbb.timing.dac_delay = 1650

## Step 9: Final front-end timing    

Finally we will optimize the front-end timing. This is done by adjusting the ADC DELAY setting on the timing panel to maximize the RMS motion of the filled bunch.
Prior to the adjustment the drive amplitude needs to be reduced until the RMS signal starts to drop, thus avoiding saturation.

Front-end design is typically quite wideband, so there might be a significant flat-top portion in the ADC response vs. clock timing.
Optimal timing in this case is midway between the points where the response starts to drop off.

In [ ]:
adc_delays = np.arange(0, 700, 40)
rms_values = bbb.sweep_adc_delay(adc_delays, wait=10, mon_type='peak')

set the maximum value:

In [ ]:
plot_sweep_results(
    adc_delays, rms_values, ctrl_label='ADC Delay', mon_label='SRAM Peak Amplitude')

In [ ]:
bbb.timing.adc_delay = 500

# Configure Feedback Loop

## Step 1: find the correct phase

In [ ]:
fb_phases = np.linspace(-180, 180, 20)
rms_values = bbb.sweep_feedback_phase(fb_phases, wait=5)

In [ ]:
plot_sweep_results(
    fb_phases, rms_values, ctrl_label='Coeff. Phase', mon_label='SRAM Peak Amplitude')

In [ ]:
bbb.feedback.edit_phase = 25 - 180
bbb.feedback.cm_edit_apply()